# Constrained positions in OpenMM

This notebook shows how to constrain the absolute positions of some atoms (i.e., lock some atoms -- in this case, backbone carbons -- to their original coordinates).

In [1]:
import openmm_scaled_md as scaled
import simtk.openmm as mm
from simtk.openmm import app
from simtk import unit

import mdtraj as md

In [2]:
pdb = app.PDBFile('AD_initial_frame.pdb')
forcefield = app.ForceField('amber96.xml', 'tip3p.xml')

pdb_system = forcefield.createSystem(pdb.topology,
                                     nonbondedMethod=app.PME, 
                                     nonbondedCutoff=1.0*unit.nanometers,
                                     constraints=None, # None due to pos constraints
                                     rigidWater=True,
                                     ewaldErrorTolerance=0.0005)

The first two cells are similar to what we already would have done to set up the system, although we can't use constraints (of the SHAKE style) in combination with absolute position constraints, so our constraints are `None`.

The next cells add positional constraints. We'll use [MDTraj's atom selection language](http://mdtraj.org/latest/atom_selection.html) to select atoms; this *does* assume that MDTraj does not change the atom ordering scheme when creating an `mdtraj.Topology` from an `openmm.app.Topology`.

In [3]:
# we use MDTraj to intelligently select the atoms we want (carbon backbone)
topology = md.Topology.from_openmm(pdb.topology)
pos_constrained_atoms = topology.select("element == 'C' and backbone")

In [4]:
# == OPTIONAL ==

# density or other info in reporters might need true mass, so we save that before changing it
true_mass = sum([pdb_system.getParticleMass(a_idx)
                 for a_idx in range(pdb_system.getNumParticles())], 
                0.0*unit.dalton)

# save individual true masses in case we want to set them back later
individual_true_masses = {a_idx: pdb_system.getParticleMass(a_idx)
                          for a_idx in pos_constrained_atoms}

In [5]:
# constrain by setting mass to zero
for a_idx in pos_constrained_atoms:
    pdb_system.setParticleMass(a_idx, 0.0*unit.dalton)

### Running with constrained absolute positions

Now we'll create a trajectory that uses this system for dynamics. Getting the `simulation` object and using it to run MD is the same as is normally done.

In [6]:
# this is equivalent to the standard BAOAB integrator, since force_scaling is 1
integrator = scaled.integrators.BAOABIntegrator(
    temperature=300.0*unit.kelvin,
    collision_rate=1.0/unit.picosecond,
    timestep=2.0*unit.femtosecond,
    force_scaling=1.0
)

In [7]:
sim = app.Simulation(pdb.topology, pdb_system, integrator)
sim.context.setPositions(pdb.positions)
sim.reporters.append(app.PDBReporter('position_constrained.pdb', 10))

In [8]:
sim.step(1000)

### Visualizing with NGLView

If you also have [NGLView](http://nglviewer.org/nglview/latest/) installed, you can visualize the trajectory in-notebook.

In [9]:
import nglview as nv
traj = md.load("./position_constrained.pdb")

In [15]:
view = nv.show_mdtraj(traj)
view.add_ball_and_stick("ACE ALA NME")
view.add_point("water and .O")
view

NGLWidget(count=100)

In this, it is clear that the 4 backbone carbons (the methyl carbon in NME, both backbone carbons in ALA, and the carbonyl carbon in ACE) stay in a fixed position, while everything else moves.